In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [3]:
try:

    url = "https://www.nngroup.com/articles/"

    source = requests.get(url)
    source.raise_for_status()

    soup = BeautifulSoup(source.text, 'html.parser')
    print(soup)

except Exception as e:
    print(e)



<!DOCTYPE html>

<html lang="en">
<head>
<script src="//media.nngroup.com/static/bundles/consent.js"></script><!-- Google Consent Mode --><script data-cookieconsent="ignore">
    window.dataLayer = window.dataLayer || [];
    function gtag() {
      dataLayer.push(arguments)
    }
    gtag("consent", "default", {
      ad_personalization: "denied",
      ad_storage: "denied",
      ad_user_data: "denied",
      analytics_storage: "denied",
      functionality_storage: "denied",
      personalization_storage: "denied",
      security_storage: "granted",
      wait_for_update: 500
    });
    gtag("set", "ads_data_redaction", true);
    gtag("set", "url_passthrough", false);
  </script><!-- End Google Consent Mode--><!-- Google Tag Manager --><script data-cookieconsent="statistics" type="text/plain">(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
                                              j=d.cre

In [4]:
data = []
counter = 1
num_pages = 10
df = pd.DataFrame()

In [5]:
while counter <= num_pages:
  url =f'https://www.nngroup.com/articles/?page={counter}'
  try:
     page = requests.get(url, timeout=10)
     time.sleep(2)
  except Exception as e:
        print(f"Failed to fetch page {counter}: {e}")
        continue
  soup = BeautifulSoup(page.text,"html.parser")
  articles = soup.find_all('li', class_='publication-list-item article')
  for article in articles:
    item={}

    articleTeaser = article.find('div', class_='article teaser')
    content = articleTeaser.find('div', class_='content')
    info = content.find('h2', class_='title')
    if info:
      item['Title'] = info.find('a').text

      timeTag = content.find('time')
      item['Publication Date'] = timeTag.find('span', class_='small large desktop only').text
      item['Description'] = content.find('p').text
      u = info.find("a").attrs["href"]
      item['Link'] = "https://www.nngroup.com"+u


      articleURL = "https://www.nngroup.com"+u

      try:
          article_page = requests.get(articleURL, timeout=10)
          s = BeautifulSoup(article_page.content, "html.parser")
          time.sleep(1)
      except Exception as e:
                print(f"Failed to fetch article {articleURL}: {e}")
                continue



      span_author = s.find('div', class_='article-authors')
      item['author'] = span_author.find('span', class_='ga-data-layer').text.strip()


      item['Article_Outline'] = []
      lst = s.find('ul', class_='toc-items')
      if lst:
           li_elements = lst.find_all('li', class_='article-table-of-content')
           for li in li_elements:
               a_tag = li.find('a')
               item['Article_Outline'].append(a_tag.text.strip())
      else:
          item['Article_Outline'] = None


      related_courses_div = s.find('div', class_='sidebar-item related-courses')
      if related_courses_div:
            h4_tag = related_courses_div.find_all('h4', class_='title heading-small')
            item['Related_Courses'] = []
            for h4 in h4_tag:
              item['Related_Courses'].append(h4.text.strip())
      else:
          item['Related_Courses'] = None


      related_topics_div = s.find('div', class_='sidebar-item related-topics')
      item['Related_Topics'] = []
      if related_topics_div:
            topics = related_topics_div.find_all('a', class_='ga-article-related-topic')
            for topic in topics:
                item['Related_Topics'].append(topic.text.strip())

      else:
          item['Related_Topics'] = None


    else:
      print('None')
    if item in data:
      continue
    else:
      data.append(item)
    print(item)
    df = pd.concat([df,pd.DataFrame([item])], ignore_index = True)
  counter +=1

{'Title': '4 Tips for Preparing AI-Enhanced Workshops', 'Publication Date': 'May 30, 2025', 'Description': 'Plan AI-enhanced workshops by preparing warmup activities involving AI, uploading context files, creating custom AIs, and providing adaptable sample prompts.', 'Link': 'https://www.nngroup.com/articles/preparing-ai-workshops/', 'author': 'Tanner Kohler', 'Article_Outline': ['1. Prepare an AI WarmUp Activity to Build Familiarity', '2. Upload Files for Quick and Easy Context', '3. Create a Custom AI for More Powerful Context and Constraints', '4. Provide Adaptable Sample Prompts', 'Conclusion'], 'Related_Courses': ['Designing AI Experiences', 'Practical AI for UX Professionals', 'Facilitating UX Workshops'], 'Related_Topics': ['Artificial Intelligence']}
{'Title': 'The Template Trap: How Template Culture Is Dumbing Down UX', 'Publication Date': 'May 30, 2025', 'Description': "Overreliance on frameworks as universal solutions rather than adaptable starting points undermines critical

In [6]:
print(df)
df.to_csv('articles.csv', index=False)

                                                Title   Publication Date  \
0          4 Tips for Preparing AI-Enhanced Workshops       May 30, 2025   
1   The Template Trap: How Template Culture Is Dum...       May 30, 2025   
2                           AI as a Creative Teammate       May 23, 2025   
3                 Designing for Serial Task Switching       May 23, 2025   
4               AI Chatbots Discourage Error Checking       May 16, 2025   
..                                                ...                ...   
65       Experience Design: The Next Iteration of UX?   December 6, 2024   
66            Tools for Unmoderated Usability Testing   December 6, 2024   
67  3 Design Processes for High Usability: Iterati...   December 3, 2024   
68  The Biggest Challenges Practitioners Encounter...  November 29, 2024   
69  UX and CX Merge: The Shift from Products to Jo...  November 29, 2024   

                                          Description  \
0   Plan AI-enhanced workshops

##**Data cleaning**

In [78]:
df = pd.read_csv("articles.csv")

In [79]:
df.head()

,Title,Publication Date,Description,Link,author,Article_Outline,Related_Courses,Related_Topics
0,4 Tips for Preparing AI-Enhanced Workshops,"May 30, 2025",Plan AI-enhanced workshops by preparing warmup...,https://www.nngroup.com/articles/preparing-ai-...,Tanner Kohler,['1. Prepare an AI WarmUp Activity to Build Fa...,"['Designing AI Experiences', 'Practical AI for...",['Artificial Intelligence']
1,The Template Trap: How Template Culture Is Dum...,"May 30, 2025",Overreliance on frameworks as universal soluti...,https://www.nngroup.com/articles/template-trap/,"Sarah Gibbons, Claire Jen","['The Template Trap', 'The Irony Is Not Lost',...","['AI for Design Workflows', 'Design Thinking E...",['Design Process']
2,AI as a Creative Teammate,"May 23, 2025",AI acts as a creative teammate in group settin...,https://www.nngroup.com/articles/ai-creative-t...,Tanner Kohler,['AI Experiment with 776 Procter & Gamble Empl...,"['Designing AI Experiences', 'Practical AI for...","['Artificial Intelligence', 'Ideation']"
3,Designing for Serial Task Switching,"May 23, 2025","Serial task switching, or rapidly shifting att...",https://www.nngroup.com/articles/serial-task-s...,Megan Chan,"['Multitasking on the Web Is a Myth', 'Impact ...","['The Human Mind and Usability', 'Persuasive a...","['Behavior Patterns', 'Psychology and UX']"
4,AI Chatbots Discourage Error Checking,"May 16, 2025",AI hallucinations threaten the usefulness of L...,https://www.nngroup.com/articles/ai-chatbots-d...,Pavel Samsonov,['Users Struggle to Verify AI Outputs Successf...,"['Designing AI Experiences', 'Practical AI for...",['Artificial Intelligence']


In [80]:
df.shape

(70, 8)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             70 non-null     object
 1   Publication Date  70 non-null     object
 2   Description       70 non-null     object
 3   Link              70 non-null     object
 4   author            70 non-null     object
 5   Article_Outline   68 non-null     object
 6   Related_Courses   65 non-null     object
 7   Related_Topics    68 non-null     object
dtypes: object(8)
memory usage: 4.5+ KB


In [82]:
df.isna().sum()

,0
Title,0
Publication Date,0
Description,0
Link,0
author,0
Article_Outline,2
Related_Courses,5
Related_Topics,2


In [83]:
if df.duplicated().any() == True:
    print('there is duplicate')
else:
    print('no duplicates')

no duplicates


In [84]:
df.dropna(inplace=True)

In [85]:
df.shape

(62, 8)

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 0 to 69
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             62 non-null     object
 1   Publication Date  62 non-null     object
 2   Description       62 non-null     object
 3   Link              62 non-null     object
 4   author            62 non-null     object
 5   Article_Outline   62 non-null     object
 6   Related_Courses   62 non-null     object
 7   Related_Topics    62 non-null     object
dtypes: object(8)
memory usage: 4.4+ KB


In [87]:
df.isna().sum()

,0
Title,0
Publication Date,0
Description,0
Link,0
author,0
Article_Outline,0
Related_Courses,0
Related_Topics,0


remove the square brackets and single quotations

In [88]:
df.head()

,Title,Publication Date,Description,Link,author,Article_Outline,Related_Courses,Related_Topics
0,4 Tips for Preparing AI-Enhanced Workshops,"May 30, 2025",Plan AI-enhanced workshops by preparing warmup...,https://www.nngroup.com/articles/preparing-ai-...,Tanner Kohler,['1. Prepare an AI WarmUp Activity to Build Fa...,"['Designing AI Experiences', 'Practical AI for...",['Artificial Intelligence']
1,The Template Trap: How Template Culture Is Dum...,"May 30, 2025",Overreliance on frameworks as universal soluti...,https://www.nngroup.com/articles/template-trap/,"Sarah Gibbons, Claire Jen","['The Template Trap', 'The Irony Is Not Lost',...","['AI for Design Workflows', 'Design Thinking E...",['Design Process']
2,AI as a Creative Teammate,"May 23, 2025",AI acts as a creative teammate in group settin...,https://www.nngroup.com/articles/ai-creative-t...,Tanner Kohler,['AI Experiment with 776 Procter & Gamble Empl...,"['Designing AI Experiences', 'Practical AI for...","['Artificial Intelligence', 'Ideation']"
3,Designing for Serial Task Switching,"May 23, 2025","Serial task switching, or rapidly shifting att...",https://www.nngroup.com/articles/serial-task-s...,Megan Chan,"['Multitasking on the Web Is a Myth', 'Impact ...","['The Human Mind and Usability', 'Persuasive a...","['Behavior Patterns', 'Psychology and UX']"
4,AI Chatbots Discourage Error Checking,"May 16, 2025",AI hallucinations threaten the usefulness of L...,https://www.nngroup.com/articles/ai-chatbots-d...,Pavel Samsonov,['Users Struggle to Verify AI Outputs Successf...,"['Designing AI Experiences', 'Practical AI for...",['Artificial Intelligence']


In [89]:
df['Article_Outline'] = df['Article_Outline'].apply(lambda x: x.strip('[]').replace("'",""))
df['Related_Courses'] = df['Related_Courses'].apply(lambda x: x.strip('[]').replace("'",""))
df['Related_Topics'] = df['Related_Topics'].apply(lambda x: x.strip('[]').replace("'",""))

In [90]:
index = df['Article_Outline'].str.contains('\\\\xa0')
print(index[index])

1     True
36    True
Name: Article_Outline, dtype: bool


In [91]:
df[df['Article_Outline'].str.contains('\\\\xa0')]

,Title,Publication Date,Description,Link,author,Article_Outline,Related_Courses,Related_Topics
1,The Template Trap: How Template Culture Is Dum...,"May 30, 2025",Overreliance on frameworks as universal soluti...,https://www.nngroup.com/articles/template-trap/,"Sarah Gibbons, Claire Jen","The Template Trap, The Irony Is Not Lost, Ampl...","AI for Design Workflows, Design Thinking Essen...",Design Process
36,Product-Specific GenAI Needs to Write for the Web,"April 4, 2025","Generative-AI outputs need to be concise, scan...",https://www.nngroup.com/articles/genai-write-f...,Taylor Dykes,"Emulate Broad-Scope AI Tools, Concision, Forma...","Designing AI Experiences, Practical AI for UX ...","Artificial Intelligence, Writing for the Web"


In [107]:
df['Article_Outline']= df['Article_Outline'].str.replace('\\xa0', ' ')

In [108]:
print(df['Article_Outline'].str.contains('\\\\xa0').any())

False


In [111]:
dfCleaned = df
dfCleaned.to_csv('cleanedArticles.csv', index=False)

##**Store the data in sql**

In [112]:
import sqlite3
dfCleaned = pd.read_csv("cleanedArticles.csv")

In [113]:
connection = sqlite3.connect('articles.db')

dfCleaned.to_sql('articles', connection, if_exists='replace')

connection.close()
